In [8]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [9]:
# -*- coding: utf-8 -*-
"""
Created on Tue Nov 19 13:12:36 2019

@author: cenkd
"""

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
from scipy.io import loadmat

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
from sklearn.metrics import precision_recall_curve
from torch.utils.tensorboard import SummaryWriter  # Added for TensorBoard

torch.manual_seed(1)

def oneofKencoder(i):
    o = torch.zeros(3)
    o[i-1] = 1
    return o

class LSTMTwitter(nn.Module):
    def __init__(self, hidden_dim, vocab_size, tagset_size):
        super(LSTMTwitter, self).__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(vocab_size, hidden_dim)
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.sigmoid = nn.Sigmoid()

    def init_hidden(self):
        return(autograd.Variable(torch.randn(1, 1, self.hidden_dim)),
               autograd.Variable(torch.randn(1, 1, self.hidden_dim)))

    def forward(self, sentence):
        model.hidden = model.init_hidden()
        feats = torch.zeros((len(sentence), 3))
        for n, w in enumerate(sentence):
            feats[n, :] = oneofKencoder(int(w))

        lstm_out, _ = self.lstm(feats.view(-1, 1, 3))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = self.sigmoid(tag_space[-1])
        return tag_scores


model = LSTMTwitter(5, 3, 1).float()
loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

# Initialize SummaryWriter
writer = SummaryWriter(log_dir=f"runs/run_1")


with open('/content/drive/MyDrive/Colab Notebooks/EE423/train_twit.txt', "r") as f:
    train_data = f.readlines()


for t in range(50):
    model.zero_grad()
    total_loss = 0

    for l_num, l in enumerate(train_data):
        wordseq = (l.rstrip()).split(' ')

        # forward pass
        y_pred = model(wordseq)

        y_train = torch.tensor([1], dtype=torch.float) if l_num <= 100 else torch.tensor([0], dtype=torch.float)

        loss = loss_fn(y_pred, y_train)
        total_loss += loss.item()
        loss.backward()


    # gradient accumulation
    optimizer.step()

    # Log the average loss for this epoch
    writer.add_scalar("Loss/train", total_loss / len(lines), t)

# Close the SummaryWriter
writer.close()


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
test_data_path = '/content/drive/MyDrive/Colab Notebooks/EE423/test_twit.txt'

with open(test_data_path, "r") as f:
    test_data = f.readlines()

# Assuming `test_data` is your test dataset
true_labels = []
predicted_probs = []

model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Disable gradient calculation
    for l_num, l in enumerate(test_data):
        wordseq = (l.rstrip()).split(' ')
        labels = torch.tensor([1], dtype=torch.float) if l_num <= 100 else torch.tensor([0], dtype=torch.float)


        # Make predictions
        prediction_prob = model(wordseq)

        # Store results
        true_labels.append(labels.item())
        predicted_probs.append(prediction_prob.item())

# At this point, `true_labels` holds the true labels, and `predicted_probs` the predicted probabilities


In [12]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Convert probabilities to binary predictions
threshold = 0.6
predicted_labels = [1 if prob > threshold else 0 for prob in predicted_probs]


# Calculate metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision, recall, f_score, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='binary')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f_score}")


Accuracy: 0.95
Precision: 0.941747572815534
Recall: 0.9603960396039604
F1-Score: 0.9509803921568628


In [13]:
# Launch TensorBoard within the notebook using the following magic command
%tensorboard --logdir=runs

Output hidden; open in https://colab.research.google.com to view.